# 스파크 고급 분석 9장

## 몬테카를로 시뮬레이션으로 금융 리스크 추정하기

In [1]:
%%configure -f
{
    "name": "mc-finance",
    "proxyUser": "hduser",
    "driverMemory": "4000M", 
    "conf": {"spark.jars.packages": "graphframes:graphframes:0.3.0-spark2.0-s_2.11",
             "spark.master": "local[2]",
             "spark.jars": "hdfs://localhost:54310/jars/ch06-lsa-2.0.0-jar-with-dependencies.jar",
             "spark.sql.crossJoin.enabled": "true"}
}

In [15]:
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import java.time.format.DateTimeFormatterBuilder
import java.io.File
import java.util.Locale
import scala.collection.mutable.ArrayBuffer
//import breeze.plot._
import org.apache.commons.math3.distribution.ChiSquaredDistribution
import org.apache.commons.math3.distribution.MultivariateNormalDistribution
import org.apache.commons.math3.random.MersenneTwister
import org.apache.commons.math3.stat.correlation.Covariance
import org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression
import org.apache.spark.sql.Dataset

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.time.LocalDate
import java.time.format.DateTimeFormatter
import java.time.format.DateTimeFormatterBuilder
import java.io.File
import java.util.Locale
import scala.collection.mutable.ArrayBuffer
import org.apache.commons.math3.distribution.ChiSquaredDistribution
import org.apache.commons.math3.distribution.MultivariateNormalDistribution
import org.apache.commons.math3.random.MersenneTwister
import org.apache.commons.math3.stat.correlation.Covariance
import org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression
import org.apache.spark.sql.Dataset


In [3]:
val DATA_HOME = "/home/hoondori/data/"
val base = "hdfs://localhost:54310/stock/"
val localBase = DATA_HOME + "stock"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DATA_HOME: String = /home/hoondori/data/
base: String = hdfs://localhost:54310/stock/
localBase: String = /home/hoondori/data/stock


# Overiew

최대손실예상액(VaR: Value at Risk)이나 조건부 VaR 최소화가 목표이다. 

VaR 계산 방법
* 분산-공분산 : 각 금융상품의 수익을 정규분포라고 가정하고 단순 계산
* 과거 시뮬레이션 : 요약 통계가 아닌 과거 데이터(금융상품별 주가 raw 그래프)의 분포로 직접 추정
* MC 시뮬레이션 - 가상 시뮬레이션을 반복해서 수행/관찰 => 확률 밀도 함수 추정 
  1. 시장 요인과 각 금융상품의 수익간의 모델링 (Linear Model)
  1. 시장 요인들의 생성 분포 정의 (fit to historical data) - 다변량 정규 분포 가정 with Non-diagonal cov
  1. 임의 시장 요인들로 구성된 실험들 제기 
  1. 각 실험에서 전체 포트폴리오의 손실 계산 => 손실에 대한 경험적 분포 정의 
  1. 손실 분포에서 n% Var 추정


## 데이터 준비 

In [4]:
def parseDouble(s: String) = try { Some(s.toDouble) } catch { case _ => None }

def readYahooHistory(file:File): Array[(LocalDate, Double)] = {
    val df = new DateTimeFormatterBuilder().
        // case insensitive to parse JAN and FEB
        parseCaseInsensitive().
        // add pattern
        appendPattern("d-MMM-yy").
        // create formatter (use English Locale to parse month names)
        toFormatter(Locale.ENGLISH)

    val lines = scala.io.Source.fromFile(file).getLines().toSeq 
    (lines.tail.map { line =>
        val cols = line.split(',')
        val date = LocalDate.parse(cols(0), df)
        (date, parseDouble(cols(1)))
    } filter { case ( date, valOpt) =>
        valOpt.isDefined
    } map { case (date, valOpt) =>
        (date, valOpt.get)
    } reverse).toArray 
}

def trimToRegion(history: Array[(LocalDate, Double)], start:LocalDate, end:LocalDate): Array[(LocalDate, Double)] = {
    
    // 기간 내의 정보만 trim
    var trimmed = history.dropWhile(_._1.isBefore(start)).takeWhile(x => x._1.isBefore(end) || x._1.isEqual(end))
    
    // 시작/끝 날짜가 없으면 fill
    if (trimmed.head._1 != start) {
        trimmed = Array((start, trimmed.head._2)) ++ trimmed
    }
    if (trimmed.last._1 != end) {
        trimmed = trimmed ++ Array((end, trimmed.last._2))
    }
    trimmed
}

def fillInHistory(history: Array[(LocalDate, Double)], start:LocalDate, end:LocalDate): Array[(LocalDate, Double)] = {
    var cur = history
    
    val filled = new ArrayBuffer[(LocalDate, Double)]
    
    var curDate = start
    while(curDate.isBefore(end)) {
        if(cur.tail.nonEmpty && cur.tail.head._1.isEqual(curDate)) { 
            cur = cur.tail
        }
        
        filled += ((curDate, cur.head._2))
        
        curDate = curDate.plusDays(1)
        
        // 주말 제외
        if (curDate.getDayOfWeek.getValue > 5) {
            curDate = curDate.plusDays(2)
        }
        
    }
    filled.toArray
}

def twoWeekReturns(history: Array[(LocalDate, Double)]): Array[Double] = {
    
    history.sliding(10).   // 10일 간격
        map { window =>
            val startPrice = window.head._2
            val endPrice = window.last._2
            ((endPrice - startPrice)) / startPrice
        }.toArray
}

def readStocksAndFactors(): (Seq[Array[Double]], Seq[Array[Double]]) = {

    val start = LocalDate.of(2009, 10, 23)
    val end = LocalDate.of(2014, 10, 23)

    // stock 확보 
    val stockDir = new File(localBase + "/stocks/")
    val files = stockDir.listFiles()
    val allStocks = files.iterator.flatMap { file =>
        try {
            Option(readYahooHistory(file))    
        } catch {
            case e: Exception => None
        }
    }
    val rawStocks = allStocks.filter(_.size >= 260*5 +10)
    val stocks = rawStocks.
        map(trimToRegion(_, start, end)).
        map(fillInHistory(_, start, end))
    
    // factor 확보
    val factorDir = localBase + "/factors/"
    val rawFactors = Array("NYSEARCA%3AGLD.csv", "NASDAQ%3ATLT.csv", "NYSEARCA%3ACRED.csv").map { x =>
        new File(factorDir + x)
    }.map { file =>
        readYahooHistory(file)
    }
    val factors = rawFactors.
        map(trimToRegion(_, start, end)).
        map(fillInHistory(_, start, end))

    val stockReturns = stocks.map(twoWeekReturns).toSeq
    val factorReturns = factors.map(twoWeekReturns).toSeq
    (stockReturns, factorReturns)
}

val (stocks, factors) = readStocksAndFactors()

// check
(stocks ++ factors).forall(_.size == stocks(0).size) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<console>:34: warning: This catches all Throwables. If this is really intended, use `case _ : Throwable` to clear this warning.
       def parseDouble(s: String) = try { Some(s.toDouble) } catch { case _ => None }
                                                                          ^
parseDouble: (s: String)Option[Double]
readYahooHistory: (file: java.io.File)Array[(java.time.LocalDate, Double)]
trimToRegion: (history: Array[(java.time.LocalDate, Double)], start: java.time.LocalDate, end: java.time.LocalDate)Array[(java.time.LocalDate, Double)]
fillInHistory: (history: Array[(java.time.LocalDate, Double)], start: java.time.LocalDate, end: java.time.LocalDate)Array[(java.time.LocalDate, Double)]
twoWeekReturns: (history: Array[(java.time.LocalDate, Double)])Array[Double]
readStocksAndFactors: ()(Seq[Array[Double]], Seq[Array[Double]])
stocks: Seq[Array[Double]] = Stream([D@3096a2a4, ?)
factors: Seq[Array[Double]] = WrappedArray(Array(0.02210526315789476, 0.0383908712890436, 0.06673

# 시장 요인 가중치 결정하기 (LinearModel Fitting)

* 각 상품별 two week returen 이 y값
* 모든 기간의 factor가 DataMatrix x 

In [5]:
// transpose
def factorMatrix(histories: Seq[Array[Double]]): Array[Array[Double]] = {
    val mat = new Array[Array[Double]](histories.head.length)
    for (i <- histories.head.indices) {
        mat(i) = histories.map(_(i)).toArray
    }
    mat
}

def featurize(factorReturns: Array[Double]): Array[Double] = {
    // x ++ x^2 ++ sqrt_x

    val squaredReturns = factorReturns.map(x => math.signum(x) * x * x)
    val squareRootedReturns = factorReturns.map(x => math.signum(x) * math.sqrt(math.abs(x)))
    squaredReturns ++ squareRootedReturns ++ factorReturns    
    
}

def linearModel(instrument: Array[Double], factorMatrix: Array[Array[Double]]): OLSMultipleLinearRegression = {
    val regression = new OLSMultipleLinearRegression()
    regression.newSampleData(instrument, factorMatrix)
    regression
}

val factorMat = factorMatrix(factors)
val factorFeatures = factorMat.map(featurize)
val factorWeights = stocks.
    map(linearModel(_, factorFeatures)).
    map(_.estimateRegressionParameters).
    filter( weight => weight(0).isNaN == false).
    toArray
factorWeights.size // NaN 이 섞인 모델링은 제거
stocks.size

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

factorMatrix: (histories: Seq[Array[Double]])Array[Array[Double]]
featurize: (factorReturns: Array[Double])Array[Double]
linearModel: (instrument: Array[Double], factorMatrix: Array[Array[Double]])org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression
factorMat: Array[Array[Double]] = Array(Array(0.02210526315789476, -0.02118465430016854, -3.9572615749893197E-4), Array(0.0383908712890436, -0.01122405760271074, 0.00405980790177256), Array(0.0667386397094906, -0.007342768968819812, 0.010596157654981116), Array(0.06433497536945813, -0.011456800504519691, 0.004737465455981089), Array(0.07638615808100664, -0.012550094916684221, 0.002857424376785952), Array(0.06674484510896118, -0.0195583596214511, -9.81932443048002E-4), Array(0.04657004830917868, -0.01758978117474617, 0.002164715143166377), Array(0.06709080393290928, -0.006612090680100708, 0.005812807881773433), Array(0.03715806180562043, 0.014742014742014694, 0.007382616399251894), Array(0.05505102518490774, 0.027673091418111

# 표본 추출

* 시장 요인 메트릭스에 대한 커널 밀도 추정 

In [6]:
// 다변량 정규 분포 fitting
val factorCov = new Covariance(factorMat).getCovarianceMatrix().getData()   // factorMat : N*3 => cov => 3x3
val factorMeans = factors.map( f => f.sum / f.size ).toArray
val factorDist = new MultivariateNormalDistribution(factorMeans, factorCov)
factorDist.sample()
factorDist.sample()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

factorCov: Array[Array[Double]] = Array(Array(9.3252388430769E-4, 5.9696785542521946E-5, 4.693902205558582E-5), Array(5.9696785542521946E-5, 5.308496396296666E-4, 1.0772004530333134E-4), Array(4.693902205558582E-5, 1.0772004530333134E-4, 5.711355671718794E-5))
factorMeans: Array[Double] = Array(0.0011934781923782765, 7.955887959746435E-4, 3.958487265604749E-4)
factorDist: org.apache.commons.math3.distribution.MultivariateNormalDistribution = org.apache.commons.math3.distribution.MultivariateNormalDistribution@50f6199f
res18: Array[Double] = Array(-0.03410139233221149, 0.052341693200974794, -0.0014622032611104714)
res19: Array[Double] = Array(0.03554517817059169, 0.004198462033487468, 0.007745118486443139)


# 실험 실행하기

* 각 trial 마다 X 발생 (from factorDist)
* factor weight를 이용해 prediction : y
* 손실 계산 
* 손실 분포 
* 손실 분포로부터 VaR 예측

In [7]:
// 동시 실험 개수 및 난수 발생으로 랜덤성 형성
val parallelism = 10
val baseSeed = 1496
val seeds = (baseSeed until baseSeed+parallelism)

//  동시 실험 개수 만큼 리파티션 : Why? seed가 겹치지 않게 해서 실험 독립을 100% 보장하기 위해
val seedDS = seeds.toDS().repartition(parallelism)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

parallelism: Int = 10
baseSeed: Int = 1496
seeds: scala.collection.immutable.Range = Range(1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505)
seedDS: org.apache.spark.sql.Dataset[Int] = [value: int]


In [12]:
// instrument : biase + weight arr
// ret : wx + b
def instrumentTrialReturn(instrument: Array[Double], trial: Array[Double]): Double = {
  
    // WX + b 계산 
    var ret = instrument(0)
    var i = 0
    while( i < trial.length) {
        ret += trial(i)*instrument(i+1)
        i += 1
    }
    
    ret
}

// 모든 금융상품을 고려한 전체 return 
def trialReturn(trial: Array[Double], instruments:Seq[Array[Double]]): Double = {
    instruments.map( instrumentTrialReturn(_, trial) ).sum / instruments.size
}

// 각 실험에서 모든 금융상품을 고려한 전체 return
//  ret = 각 실험별 리턴
def trialReturns(seed:Long, numTrials:Int, instruments:Seq[Array[Double]], 
                 factorMeans: Array[Double], factorCov:Array[Array[Double]]): Seq[Double] = {
    
    // 다변량 분포 fitting : 각 spark job 마다 다시 특정 seed로부터 순환 주기가 긴 seed generator(MT)를 사용해 실험 독립 보장
    val rand = new MersenneTwister(seed)
    val factorDist = new MultivariateNormalDistribution(rand, factorMeans, factorCov)
    
    val trialReturns = new Array[Double](numTrials)

    for (i <- 0 until numTrials) {
        val sample = factorDist.sample()
        val feature = featurize(sample)
        trialReturns(i) = trialReturn(feature, instruments)
    }
    trialReturns
}


val numTrials = 100000
val trials = seeds.flatMap(trialReturns(_, numTrials / parallelism, factorWeights, factorMeans, factorCov))
val trialsDS = trials.toDS().cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

instrumentTrialReturn: (instrument: Array[Double], trial: Array[Double])Double
trialReturn: (trial: Array[Double], instruments: Seq[Array[Double]])Double
trialReturns: (seed: Long, numTrials: Int, instruments: Seq[Array[Double]], factorMeans: Array[Double], factorCov: Array[Array[Double]])Seq[Double]
numTrials: Int = 100000
trials: scala.collection.immutable.IndexedSeq[Double] = Vector(0.04976403826352305, 0.013224905111413231, 0.01897621299663793, 0.027623531244437113, 0.0026007128847401716, 0.0225436478640878, 0.06048547791773244, 0.040546359361759016, 0.02738448199064011, 0.01144572655993988, 0.04467572063597882, 0.017646599629715872, 0.0022815165918509146, 0.03901623218222861, 0.021567813975053265, 0.011557712834543198, 0.02362046533281404, 0.03453827677770612, 0.010583873501656132, 0.008307565581656174, 0.014592237545986446, 0.012243887917473146, 0.0015932236783471588, 0.03685604323863246, -0.002769623873547922, 0.031094936857761202, -0.036667952517917345, 0.01279695990127956, 0.0

## 실험 결과의 (정규) 분포 에서 5% quantile 이하값을 VaR 로 한다

In [27]:
def fivePercentVaR(trials:Dataset[Double]): Double = {
    trials.stat.approxQuantile("value", Array(0.05), 0.0).head
}

val valueAtRisk = fivePercentVaR(trialsDS)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fivePercentVaR: (trials: org.apache.spark.sql.Dataset[Double])Double
valueAtRisk: Double = -0.021429430793046003


### Conditional VaR 은 하위 5% 들의 평균으로 결정

In [28]:
def fivePercentCVaR(trials:Dataset[Double]): Double = {
    val topLosses = trials.orderBy("value").limit(math.max(trials.count.toInt/20, 1))
    topLosses.agg("value" -> "avg").first()(0).asInstanceOf[Double]
}

val conValueAtRisk = fivePercentCVaR(trialsDS)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fivePercentCVaR: (trials: org.apache.spark.sql.Dataset[Double])Double
conValueAtRisk: Double = -0.03189059064494651


## 통계량(VaR)의 신뢰구간 측정 (Bootstrap Interval)

* bootstrap distribution
  * sample로부터 복원추출(sample with replacement, 동일크기)해서 얻은 복수 개의 resample들 확보
  * resample로부터의 통계(ex. VaR) 추출 
* bootstrap interval
  * 얻은 추출치들에서 interval 계산  (ex. 5% 이면 양쪽 2.5% 제외한 구간)

In [30]:
def bootstrapInteval(
    trials: Dataset[Double],
    computeStatistics: Dataset[Double] => Double,
    numResamples: Int, 
    prob: Double
): (Double, Double) = {
    
    val stats = (0 until numResamples).map { i =>
        val resample = trials.sample(true, 1.0)  // 복원 전수 추출
        computeStatistics(resample)
    }
    
    val lowerIndex = (numResamples * prob / 2 - 1).toInt
    val upperIndex = math.ceil(numResamples * (1 - prob/2)).toInt
    
    ( stats(lowerIndex), stats(upperIndex) )
}

bootstrapInteval(trialsDS, fivePercentVaR, 100, 0.05)
bootstrapInteval(trialsDS, fivePercentCVaR, 100, 0.05)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bootstrapInteval: (trials: org.apache.spark.sql.Dataset[Double], computeStatistics: org.apache.spark.sql.Dataset[Double] => Double, numResamples: Int, prob: Double)(Double, Double)
res65: (Double, Double) = (-0.02162347815202497,-0.02144372937173182)
res66: (Double, Double) = (-0.031554605723052076,-0.03177225802643956)
